In [ ]:
import os 
import requests
from bs4 import BeautifulSoup
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

import re


In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
my_activeloop_org_id = "charanvardhan"
my_activeloop_dataset_name = "VoiceAssistant-embeddings"
dataset_path = 'hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}'

embeddings = OpenAIEmbeddings(model_name="text-embedding-ada-002")

In [12]:
def get_documents_url(url):
    return [
    '/docs/huggingface_hub/guides/overview',
    '/docs/huggingface_hub/guides/download',
    '/docs/huggingface_hub/guides/upload',
    '/docs/huggingface_hub/guides/hf_file_system',
    '/docs/huggingface_hub/guides/repository',
    '/docs/huggingface_hub/guides/search',
    ]

def construct_url(base_url, relative_paath):
    return base_url + relative_paath

In [14]:
def scrape_page_content(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract the desired content from the page 
    text = soup.body.text.strip()

    # remove none ascii characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def scrape_all_content(base_url, relative_paths, 
                       filename):
    all_text = []
    for path in relative_paths:
        url = construct_full_url(base_url, path)
        text = scrape_page_content(url)
        all_text.append(text)
    

    with open(filename, 'w') as f:
        f.write('\n'.join(all_text))
        
    return all_text